In [ ]:
import geopandas as gpd
import pandas as pd
import xarray as xr
import rioxarray
import xvec
import os

from dask.distributed import Client, LocalCluster
import dask

from utils import download_and_read_fwi_data_for_dates

## Constants and Dask Set Up

In [ ]:
mtbs_raster_data_dir = "./data/mtbs_rasters"
mtbs_perimeter_data_path = "./data/mtbs_perimeters/mtbs_perims_DD.shp"
eia_utility_service_areas_path = "./data/eia_service_areas/Electric_Retail_Service_Territories.geojson"
fwi_variable = "GPM.LATE.v5_FWI"
memory_per_worker = "30GB"
n_workers = 6

CRS = "EPSG:4269"

In [ ]:
cluster = LocalCluster(
    n_workers=n_workers,
    threads_per_worker=2,  
    memory_limit=memory_per_worker  
)

client = Client(cluster)

## Step 1.

First, we read in the MTBS Wildfire and FWI data and package them in a list of dictionaries for processing. 

MTBS Data - https://www.mtbs.gov/direct-download

FWI Data - https://portal.nccs.nasa.gov/datashare/GlobalFWI/v2.0/fwiCalcs.GEOS-5/Default/GPM.LATE.v5/



In [ ]:

mtbs_raster_files = os.listdir(mtbs_raster_data_dir)

mtbs_gdf = gpd.read_file(mtbs_perimeter_data_path)

input_data = []

for file in mtbs_raster_files:
    data_struct = {}

    file_info = file[:-4].split("_")
    state = file_info[1]
    year = int(file_info[2])

    data_struct["state"] = state
    data_struct["year"] = year

    da_mtsb_raster = rioxarray.open_rasterio(f"{mtbs_raster_data_dir}/{file}")
    da_mtsb_raster = da_mtsb_raster.rio.reproject(CRS)
    # We filter here to include only values of meaningful fire severity
    da_mtsb_raster = da_mtsb_raster.where((da_mtsb_raster >= 2) & (da_mtsb_raster < 5))
    da_mtsb_raster.name = "fire_severity"
    
    # Read in historic fire perimeters
    gdf_filtered = (mtbs_gdf.loc[(mtbs_gdf["Incid_Type"]=="Wildfire") & (mtbs_gdf["Ig_Date"].dt.year == data_struct["year"]) & (mtbs_gdf["Event_ID"].str.startswith(data_struct["state"]))])
    gdf_temp = gdf_filtered.set_index("Event_ID")[["geometry"]]
    gdf_temp = gdf_temp.to_crs(da_mtsb_raster.spatial_ref.crs_wkt)
    
    # Read in NetCDF Fire Weather Index data from NASA. This download will take some time
    ds_fwi = download_and_read_fwi_data_for_dates(gdf_filtered["Ig_Date"])
    ds_fwi = ds_fwi.rio.write_crs("EPSG:4326")
    ds_fwi = ds_fwi.rename({"lat": "y", "lon": "x", fwi_variable: "fwi"})
    ds_fwi = ds_fwi.sel(x=slice(-125.4, -112.5), y=slice(32.4, 50.1))
    da_fwi = ds_fwi["fwi"].compute()
    da_fwi_reproj = da_fwi.rio.reproject_match(da_mtsb_raster)
    bounds = da_mtsb_raster.rio.bounds()
    da_fwi_reproj_clipped = da_fwi_reproj.rio.clip_box(*bounds)

    # Add copies of each processed input dataset
    data_struct["da_fire_severity"] = da_mtsb_raster.copy()
    data_struct["gdf"] = gdf_temp.copy()
    data_struct["da_fwi"] = da_fwi_reproj_clipped.copy()

    print(f"{state} {year} processed!")
    input_data.append(data_struct)


## Step 2.

We zonally aggregate the data to get the fire severity (mean, max) of each occurence of wildfire, as well as the Fire Weather Index (mean, max) measured at the ignition date of the fire. 

Note, we only have data for the ignition date. We do not know how long each fire burned.



In [ ]:
fire_severity_dfs = []

for data in input_data:
    da_name = data["da_fire_severity"].name
    da_temp = data["da_fire_severity"].xvec.zonal_stats(
        data["gdf"].geometry,
        x_coords="x",
        y_coords="y",
        stats=["mean", "max"],
        method="exactextract",
        index=True,
    )
    da_temp.name = da_name
    df_temp = da_temp.to_dataframe().reset_index()

    df_mean = df_temp.loc[df_temp["zonal_statistics"]=="mean"]
    df_mean = df_mean.rename(columns={da_name: da_name+"_mean"})

    df_max = df_temp.loc[df_temp["zonal_statistics"]=="max"]
    df_max = df_max.rename(columns={da_name: da_name+"_max"})


    df_temp =df_mean.merge(df_max, how="inner", on="Event_ID")[["Event_ID", da_name+"_mean", da_name+"_max"]]
    fire_severity_dfs.append(df_temp.copy())

    state = data["state"]
    year = data["year"]
    print(f"{state} {year} fire severity processed!")

fwi_dfs = []

for data in input_data:
    da_name = data["da_fwi"].name
    da_temp = data["da_fwi"].xvec.zonal_stats(
        data["gdf"].geometry,
        x_coords="x",
        y_coords="y",
        stats=["mean", "max"],
        method="exactextract",
        index=True,
    )
    da_temp.name = da_name
    df_temp = da_temp.to_dataframe().reset_index()

    df_mean = df_temp.loc[df_temp["zonal_statistics"]=="mean"]
    df_mean = df_mean.rename(columns={da_name: da_name+"_mean"})

    df_max = df_temp.loc[df_temp["zonal_statistics"]=="max"]
    df_max = df_max.rename(columns={da_name: da_name+"_max"})

    df_temp = pd.merge(df_max, df_mean, how="left", on=["Event_ID", "time"])[["Event_ID", "time", da_name+"_mean", da_name+"_max"]]

    fwi_dfs.append(df_temp.copy())

    state = data["state"]
    year = data["year"]
    print(f"{state} {year} FWI processed!")

fire_severity_df = pd.concat(fire_severity_dfs)
fwi_df = pd.concat(fwi_dfs)
fire_severity_df = fire_severity_df.drop_duplicates(subset=["Event_ID"])
fwi_df = fwi_df.drop_duplicates(subset=["Event_ID", "time"])

fire_df = fwi_df.merge(fire_severity_df, how='left', on="Event_ID")

mtbs_gdf_merged = mtbs_gdf.merge(fire_df, how='left', left_on=["Event_ID", "Ig_Date"], right_on=["Event_ID", "time"])


## Step 3

We read in the utility service area data and join the fire metrics with it. Our final output contains each utility service area for Oregon, California, and Washington and every wildfire that occured in their service area. The fire is quantified by its severity and fire weather index value. 

EIA Utility Service Area Data - https://atlas.eia.gov/maps/geoplatform::electric-retail-service-territories-2

In [ ]:
service_areas_raw = gpd.read_file(eia_utility_service_areas_path)
service_areas = service_areas_raw.to_crs(CRS)

mtbs_gdf_final = mtbs_gdf_merged[["Event_ID", "Ig_Date", "BurnBndAc", "fire_severity_mean", "fire_severity_max", "fwi_mean", "fwi_max", "geometry"]]

df_final = service_areas.sjoin(mtbs_gdf_final)
df_final = df_final.drop(columns=["geometry", "index_right"])
df_final = df_final.dropna(subset="fire_severity_mean")
df_final = df_final.rename({"Event_ID": "Fire_ID"})
df_final = df_final.sort_values(by=["ID", "Ig_Date"])

df_final.to_csv("utility_service_area_fire_metrics.csv.gz", index=False, compression="gzip")